### 대표값으로 대체

- 가장 널리 사용되는 방법
- 사용하기 부적절한 경우
    1. 소수 피처에 결측이 쏠린 경우
    2. 피처 간에 상관성이 아주 큰 경우

### sklearn의 전처리 모델 : Proprocessing model
- 평가 데이터는 전처리 모델을 학습하는데 사용하지 않음

#### sklearn.impute.SimpleImputer

- 결측이 있는 변수의 대표값으로 결측을 대체하는 인스턴스
- 변수 타입에 따라 두개의 인스턴스를 같이 적용해야 할 수 있음
- 문법
    - class sklearn.impute.SimpleImputer(*, missing_values=nan, strategy='mean', fill_value=None, verbose=0, copy=True, add_indicator=False)
    - strategy= default=’mean’
        - 'most_frequent', 'median'


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

#그래프 패키지 
import matplotlib.pyplot as plt 
%matplotlib inline 
# 한글 문제 해결
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system...  sorry~~~')# 한글 문제 해결
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system...  sorry~~~')


# 동시 출력
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

#### 1. 모든 피처의 데이터 타입이 연속형인 경우

In [14]:
df = pd.read_csv("./data/cleveland.csv")

In [3]:
X = df.drop('Output', axis = 1)
Y = df['Output']

In [4]:
X.head()

,Age,Sex,Cp,Trestbps,Chol,Fbs,Restecg,Thalach,Exang,Oldpeak,Slope,Ca,Thal
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0
1,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0
2,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0
3,56,1,2,120,236,0,0,178,0,0.8,1,0.0,3.0
4,57,0,4,120,354,0,0,163,1,0.6,1,0.0,3.0


In [5]:
# 학습 데이터와 평가 데이터로 분리
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y,random_state=4)

In [6]:
Train_X.head()

,Age,Sex,Cp,Trestbps,Chol,Fbs,Restecg,Thalach,Exang,Oldpeak,Slope,Ca,Thal
179,44,1,3,120,226,0,0,169,0,0.0,1,0.0,3.0
26,61,1,3,150,243,1,0,137,1,1.0,2,0.0,3.0
89,43,0,3,122,213,0,0,165,0,0.2,2,0.0,3.0
215,35,1,2,122,192,0,0,174,0,0.0,1,0.0,3.0
146,56,1,1,120,193,0,2,162,0,1.9,2,0.0,7.0


In [7]:
# 결측치 확인
Train_X.isnull().sum()

# 결측치가 많지 않음 - 학습 데이터로만 보면 데이터 삭제(행)할 수도 있지만, 새로 들어올 데이터에 결측이 있을 수 도 있다는 가정을 함
# 소수 트정 여에 결측의 쏠림은 없으

Age         0
Sex         0
Cp          0
Trestbps    0
Chol        0
Fbs         0
Restecg     0
Thalach     0
Exang       0
Oldpeak     0
Slope       0
Ca          2
Thal        1
dtype: int64

In [8]:
# 관련성 확인 - 관련성이 없음 : 대표값 대체 활용 가능
# 주의 : 모든 변수가 연속형이므로 가능한 확인
Train_X.corr().sum()/(len(Train_X.columns) )


Age         0.171026
Sex         0.095620
Cp          0.085150
Trestbps    0.128941
Chol        0.104358
Fbs         0.090858
Restecg     0.136377
Thalach    -0.091495
Exang       0.146873
Oldpeak     0.189197
Slope       0.133880
Ca          0.153850
Thal        0.193083
dtype: float64

In [10]:
# 대표값 이용 결측치 대체
from sklearn.impute import SimpleImputer

# 인스턴스 생성
SI = SimpleImputer(strategy = 'mean')

# 학습
SI.fit(Train_X)
SI.transform(Train_X)

SimpleImputer()

array([[44.,  1.,  3., ...,  1.,  0.,  3.],
       [61.,  1.,  3., ...,  2.,  0.,  3.],
       [43.,  0.,  3., ...,  2.,  0.,  3.],
       ...,
       [66.,  1.,  4., ...,  1.,  0.,  6.],
       [67.,  0.,  4., ...,  1.,  2.,  3.],
       [56.,  1.,  4., ...,  2.,  1.,  6.]])

In [11]:
# sklearn instance의 출력은 ndarray이므로 다시 DataFrame으로 바꿔줌
Train_X = pd.DataFrame(SI.transform(Train_X), columns = Train_X.columns)

In [12]:
# Test_X는 fit을 진행하지 않음
Test_X = pd.DataFrame(SI.transform(Test_X), columns = Test_X.columns)

In [13]:
Train_X.isnull().sum()

Age         0
Sex         0
Cp          0
Trestbps    0
Chol        0
Fbs         0
Restecg     0
Thalach     0
Exang       0
Oldpeak     0
Slope       0
Ca          0
Thal        0
dtype: int64

#### 2. 피처간 데이터 타입이 다른 경우

In [15]:
df = pd.read_csv("./data/saheart.csv")
#Chd : 관상도맥질환

In [16]:
X = df.drop('Chd', axis = 1)
Y = df['Chd']

In [17]:
# 학습 데이터와 평가 데이터로 분리
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_y, Test_y = train_test_split(X, Y, random_state=4)


In [18]:
Train_X.shape

(346, 9)

In [19]:
# 결측치 확인
Train_X.isnull().sum()

Sbp          0
Tobacco      0
Ldl          0
Adiposity    0
Typea        0
Obesity      6
Alcohol      4
Age          0
Famhist      5
dtype: int64

In [20]:
Train_X['Famhist'].value_counts() #범주형 변수로 판단됨

0.0    204
1.0    137
Name: Famhist, dtype: int64

In [21]:
Train_X.corr().sum()/len(Train_X.columns)

# 특징간 관계가 크지 않음

Sbp          0.282538
Tobacco      0.194214
Ldl          0.280358
Adiposity    0.400747
Typea        0.109919
Obesity      0.339761
Alcohol      0.198943
Age          0.349853
Famhist      0.238963
dtype: float64

In [22]:
# Famhist : 범주형 변수
# 그외 변수 : 연속형 변수 (평균)

# 대표값을 평균으로 할지, 최빈값을 사용할지 결정할 수 없음 => 데이터를 분할해서 사용

Train_X_cate = Train_X[['Famhist']]
Train_X_cont = Train_X.drop('Famhist', axis=1)

Test_X_cate = Test_X[['Famhist']]
Test_X_cont = Test_X.drop('Famhist', axis=1)

In [23]:
# 대표값을 활용한 결측치 대체

from sklearn.impute import SimpleImputer

#각 데이터에 사용할 인스턴스 생성
SI_mode = SimpleImputer(strategy='most_frequent') # 대표값 최빈값
SI_mean = SimpleImputer(strategy='mean') #대표값 평균

In [24]:
#학습
SI_mode.fit(Train_X_cate)
SI_mean.fit(Train_X_cont)

SimpleImputer(strategy='most_frequent')

SimpleImputer()

In [25]:
# arr -> df 로 변환, 결측치 대체
# sklearn instance의 출력은 ndarray이므로 다시 DataFrame으로 바꿔줌
Train_X_cate = pd.DataFrame(SI_mode.transform(Train_X_cate),
                            columns = Train_X_cate.columns)

Test_X_cate = pd.DataFrame(SI_mode.transform(Test_X_cate),
                           columns = Test_X_cate.columns)

Train_X_cont = pd.DataFrame(SI_mean.transform(Train_X_cont),
                            columns = Train_X_cont.columns)

Test_X_cont = pd.DataFrame(SI_mean.transform(Test_X_cont),
                           columns = Test_X_cont.columns)


In [26]:
# 분할된 df 결합
Train_x = pd.concat([Train_X_cate, Train_X_cont], axis=1)
Test_x = pd.concat([Test_X_cate, Test_X_cont], axis=1)

In [27]:
Train_X.isnull().sum()
Test_X.isnull().sum()

Sbp          0
Tobacco      0
Ldl          0
Adiposity    0
Typea        0
Obesity      6
Alcohol      4
Age          0
Famhist      5
dtype: int64

Sbp          0
Tobacco      0
Ldl          0
Adiposity    0
Typea        0
Obesity      4
Alcohol      2
Age          0
Famhist      1
dtype: int64